# CSS selector
## 원티드
- recruit_position : `.JobHeader_className__HttDA h2`
- is_hiring : `button.ApplyBtn_button__BT821`
- recruit_thumb : `.JobImage_JobImage__slide__w4mSW img`
- recruit_url 

## 사람인
- recruit_position : `.tit_job`(문장형)
- is_hiring : `.wrap_jv_header .btn_apply a` 첫번째거.
	- 홈페이지 지원 : `.sri_btn_homepage_apply`
	- 일반 지원 : `.sri_btn_immediately`
	- 마감됨 : `.sri_btn_expired_apply`
- recruit_thumb : X
- recruit_url 

## 잡코리아
- recruit_position : `div.recruitment > table td.header`
- is_hiring : `div.sumBtn button` "title" attribute
- recruit_thumb : X
- recruit_url 

In [14]:
import requests as req
from bs4 import BeautifulSoup as bs

In [15]:
keyword = '무신사'

V_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
SAVE_PATH = './datas/recruit'

## 사람인

In [66]:
# 사람인은 파라미터가 좀 많다.
SARAMIN_HOME = 'https://www.saramin.co.kr'
SARAMIN_SEARCH = SARAMIN_HOME + '/zf_user/search?search_area=main&search_done=y&search_optional_item=n&searchType=recently&searchword={}'

soup = bs(req.get(SARAMIN_SEARCH.format(keyword, 1), headers={'user-agent': V_AGENT}).text, 'lxml')

# 회사명 검사
urls = []
first_corp_name = soup.select_one('.corp_name b').text

cont_selector = '#recruit_info_list > .content > div'
corp_name_selector = cont_selector + ':nth-child({}) > .area_corp b'
atag_selector = cont_selector + ':nth-child({}) h2.job_tit a'

## 잡코리아

In [78]:
JOBKR_HOME = 'https://www.jobkorea.co.kr'
JOBKR_SEARCH = JOBKR_HOME + '/Search/?stext={}'

soup = bs(req.get(JOBKR_SEARCH.format(keyword, 1), headers={'user-agent': V_AGENT}).text, 'lxml')

# 회사명 검사
urls = []
first_corp_name = soup.select_one('.post-list-corp strong').text

cont_selector = '#content div.cnt-list-wrap .recruit-info > .lists .list-default li'
corp_name_selector = cont_selector + ':nth-child({}) .post-list-corp strong'
atag_selector = cont_selector + ':nth-child({}) .post-list-info a'

## 사람인 & 잡코리아 공통 로직

In [81]:
# 검색 결과가 있는 경우에만 공고 링크 따기
if first_corp_name is not None:
    
    # 있는 채용공고 만큼 돌린다.
    for i in range(len(soup.select(cont_selector))):
        corp_name_selector = corp_name_selector.format((i+1), 1)
        atag_selector = atag_selector.format((i+1), 1)
        
        # 만료한 경우
        if soup.select_one(corp_name_selector) is None:
            break
        
        # 최초 검색된 기업명과 검색 기업명이 상이해진 경우
        elif first_corp_name == soup.select_one(corp_name_selector).text:
            urls.append(JOBKR_HOME + soup.select_one(atag_selector)['href'])
urls

['https://www.jobkorea.co.kr/Recruit/GI_Read/42171104?Oem_Code=C1&logpath=1&stext=무신사&listno=1',
 'https://www.jobkorea.co.kr/Recruit/GI_Read/42171104?Oem_Code=C1&logpath=1&stext=무신사&listno=1',
 'https://www.jobkorea.co.kr/Recruit/GI_Read/42171104?Oem_Code=C1&logpath=1&stext=무신사&listno=1',
 'https://www.jobkorea.co.kr/Recruit/GI_Read/42171104?Oem_Code=C1&logpath=1&stext=무신사&listno=1',
 'https://www.jobkorea.co.kr/Recruit/GI_Read/42171104?Oem_Code=C1&logpath=1&stext=무신사&listno=1',
 'https://www.jobkorea.co.kr/Recruit/GI_Read/42171104?Oem_Code=C1&logpath=1&stext=무신사&listno=1',
 'https://www.jobkorea.co.kr/Recruit/GI_Read/42171104?Oem_Code=C1&logpath=1&stext=무신사&listno=1',
 'https://www.jobkorea.co.kr/Recruit/GI_Read/42171104?Oem_Code=C1&logpath=1&stext=무신사&listno=1',
 'https://www.jobkorea.co.kr/Recruit/GI_Read/42171104?Oem_Code=C1&logpath=1&stext=무신사&listno=1',
 'https://www.jobkorea.co.kr/Recruit/GI_Read/42171104?Oem_Code=C1&logpath=1&stext=무신사&listno=1',
 'https://www.jobkorea.co.kr/R

## 원티드

In [ ]:
import requests as req
import pandas as pd

import json

def get_recruit_info(keyword):
    WANTED_SEARCH ='https://www.wanted.co.kr/api/chaos/search/v1/results?1691048648606=&query={}&country=kr&job_sort=company.response_rate_order&years=-1&locations=all'
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
        'Referer': f'https://www.wanted.co.kr/search?query={keyword.encode("UTF-8").decode("iso-8859-1")}'
    }
    result = json.loads(req.get(WANTED_SEARCH.format(keyword, 1), headers=headers).text)

    result = result['positions']['data']
    
    return pd.DataFrame({
        'recruit_uid': [{el['id']} for el in result],
        'recruit_url': [f"https://www.wanted.co.kr/wd/{el['id']}" for el in result],
        'recruit_position': [el['position'] for el in result],
        'recruit_thumb': [el['title_img']['origin'] for el in result]
    })
